# Streaming Technologies 
Streaming is useful for realtime analytics, online machine learning, continuous computation, and more.

## Apache Kafka
*A distributed, partitioned, replicated commit log service.*     

Data streams are partitioned and spread over a cluster of machines to allow data streams larger than the capability of any single machine and to allow clusters of coordinated consumers
![kafka](images/kafka_producer_consumer.png)

- Kafka maintains feeds of messages in categories called topics.
- Processes that publish messages to a Kafka topic: *producers.*
- Processes that subscribe to topics and process the feed of published messages: *consumers*
- Kafka is run as a cluster comprised of one or more servers, each of which is called a *broker.*
- Java client for communication between clients and servers is provided natively; other languages are available.

#### Useful for:
- Website activity tracking
- Metrics in operational monitoring, i.e. aggregating stats from distributed applications
- Log aggregation, collecting the physical log files off servers and putting them in a central place (e.g. HDFS) for processing
- Stream processing

## Apache Storm
*"A free and open source distributed realtime computation system. Storm makes it easy to reliably process unbounded streams of data, doing for realtime processing what Hadoop did for batch processing."*

- Fast: can process 1 million+ tuples per second per node!
- Scalable and fault-tolerant
- Easy to set up and operate

#### Abstractions 
- **Core abstraction:** The *stream*, an unbounded sequence of tuples. Storm provides the primitives for transforming a stream into a new stream in a distributed and reliable way. 
- **Spout:** a source of streams, e.g. a connection to the Twitter API
- **Bolt:** consumes any number of input streams and runs processing logic; possibly emits new streams. 
    - E.g. run functions, filter, aggregate streams, join streams, talk to databases, etc.
- Networks of spouts and bolts are packaged into a **topology**, which is the top level abstraction that you submit to Storm clusters for execution. 
![storm](images/storm_topology.png)

- Each node in the topology executes in parallel (you can specify the level of parallelism for each node) 

## Spark Streaming

### Core abstraction:
The `DStream`, aka discretized stream - a sequence of data arriving over time.
- Internally a `DStream[T]` is represented as a sequence of `RDD[T]`s representing the elements `T` that arrive at each time step
- A `DStream` can be created from various input sources, e.g. Kafka, HDFS
- A `DStream` supports two types of operations: 
    - *Transformations* yield a new DStream. Many of the same operations available on RDDS, in addition to operations related to time e.g. sliding windows
    - *Output operations* write data to an external system 
    
Many functions abstract over the `RDD` layer.  The function

- `flatMap`
- `map`
- `filter`
- `reduce`
- `union`

all operate directly on elements `T`.  Other functions, most notably 

- `foreachRDD`
- `transform`

depend explicitly on the `RDD` level.  Remember that while the elements of the `DStream` are separted by time (some happen earlier than others), the elements of individual `RDD`s are separated by "distance" (they are potentially on separate nodes).  Finally, there are operations that aggregate in time, notably

- `slice`: (selects a "small" subset of the `DStream` by time)
- `window`: creates a new `DStream` generated by sliding windows of the previous ones.

## Building a Spark Streaming application
#### Imports
The most basic spark applications need to import this code:
```scala
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkConf
import org.apache.spark.streaming.{Seconds, StreamingContext}
```

#### Creating a spark context
Inside our spark app, the first step is to setup a new `StreamingContext`.  This functions much like a usual `SparkContext` but takes an additional `duration` argument which sets how often batches are generated.  In the example below, we are aiming for a second.  We also reduce the warning level to be less loquacious so we can actually see the output.

```scala
val sparkConf = new SparkConf().setAppName("QueueStream")
val ssc = new StreamingContext(sparkConf, Seconds(1))

// Set warning level to not get as many qarnings
Logger.getRootLogger.setLevel(Level.WARN)
```

#### Extra build dependencies: `build.sbt`:
```scala
libraryDependencies += "org.apache.spark" %% "spark-streaming" % "1.3.0" % "provided"
```

#### Building
Build the project using this bash command from the command prompt

```bash
sbt assembly  # or `sbt clean assembly`
```

### A simple example

and run it using

```bash
spark-submit target/scala-*/streaming-example-assembly-*.jar -p Netcount --host localhost --port 9999
```

to start this spark app listening on port 9999.  Then in another terminal, start

```bash
nc -lk 9999
```

and type in lines to have them broadcasted on port 9999.  You should see the word counts show up in the output of our spark program.

### How it works

Inside your spark app, the first step is to setup a new `StreamingContext`.  This functions much like a usual `SparkContext` but takes an additional `duration` argument which sets how often batches are generated.  In the example below, we are aiming for a second.  We also reduce the warning level to be less loquacious so we can actually see the output.

```scala
    val sparkConf = new SparkConf().setAppName("QueueStream")
    val ssc = new StreamingContext(sparkConf, Seconds(1))
    
    // Set warning level to not get as many qarnings
    Logger.getRootLogger.setLevel(Level.WARN)
```

Next, we create a socket text `DStream` (a `DStream` whose data comes from the url and port numbers specified below).  We run a standard wordcount on them and print the results, line-by-line.

```scala
// Create the Socket Text Stream and use it do some processing
val wordCounts = ssc.socketTextStream(args(0), args(1).toInt)
                      .flatMap(_.split(" "))
                      .map(x => (x, 1))
                      .reduceByKey(_ + _)

wordCounts.print()
```

All the operations above were declarative.  We need to start the computation and wait until the execution stops or an exception is called.

```scala
ssc.start()
ssc.awaitTermination()
```

You can find access to the code at [projects/streaming-example/src/main/scala/streamingexample/Netcount.scala](projects/streaming-example/src/main/scala/streamingexample/Netcount.scala)

## Keeping track of state

We do a more complex example which keeps track of state in between operations.  In this project, we analyze the word frequency of Hamlet's famous soliloquy.  In the main function, the setup requires a checkpoint directory where our intermediate states are persisted for safe keeping:

```scala
ssc.checkpoint("/tmp/")   // set checkpoint
```

Next, we create a queue populated with lines of shakespeare to be streamed:
```scala
val rddQueue = Queue[RDD[String]](shakespeare
  .map(line => ssc.sparkContext.makeRDD(Seq(line))): _*
)

val wordStream = ssc.queueStream(rddQueue, oneAtATime=true)
                      .flatMap(_.split(" "))
                      .map( x => (x, 1))
                      .reduceByKey(_ + _)
```

Finally, we need update our state (by key).  Firstly, let's specify the update function:

```scala
def updateFunc(newValues: Seq[Int], runningCount: Option[Int]): Option[Int] = {
Some(runningCount.getOrElse(0) + newValues.sum)
}
```

and then build it within the main function:

```scala
val stateDstream = wordStream.updateStateByKey[Int](updateFunc _)
```

(For each key) `updateFunc` takes the `runningCount` and updates it wit hthe sum of counts coming in.  Notice that what `updateStateByKey` does across time, `reduceByKey` does across an `RDD`: propagating a summary state.

You can find a copy of the source code at 
[projects/streaming-example/src/main/scala/streamingexample/Shakespeare.scala](projects/streaming-example/src/main/scala/streamingexample/Shakespeare.scala), 

### Window state

Finally, we can increase the "duration" of the "window" over which `DStream`'s `RDD`s data read from.  Remember that each `DStream` is actually a stream of `RDD`s.  In our examples thus far, the windows arrive secondly and represent the data in the last second.  Both of those can be retweaked.

![image](images/streaming-dstream-window.png)

For example, in the above diagram, we have created a new `DStream` based on the previous `DStream` but where each window represents the last 3 (units of time) and this occurs every 2 (units of time).  That is, we have created a window where
- *window length* -- duration or the window (3 in this example)
- *sliding interval* -- interval at which the window is outputted (2 in this example).

### An example
In our example, we first create an original stream from a queue.  Each element of the stream contains an `RDD` holding a single integer.

```scala
val rddQueue = Queue[RDD[Int]]((1 to 100)
  .map( i => ssc.sparkContext.makeRDD(Seq(i))): _*
)

val original = ssc.queueStream(rddQueue, oneAtATime=true).map(_.toDouble)
```

We create two streams from this original stream, the first one just reflects the current `RDD`, the second one reflects the second one reflects the sum of the last two RDDs.
```scala
val value = original.map(x => ("value" -> x))

val lastTwoSum = original.window(Seconds(2))
  .transform( rdd => ssc.sparkContext.makeRDD(Seq(rdd.sum)) )
  .map(x => ("lastTwoSum" -> x))
```

Finally, we join them using a union and print those values to the screne.
```scala
val output = ssc.union(Seq(value, lastTwoSum)).print()
```

You'll notice that in the new stream, we see the values for boht "value" and "lastTwoSum" and they are as promised.

### Streaming tweets demo

The project lives here: `datacourse/projects/streaming_twitter_cl`

#### Collecting tweets with built-in Twitter utilities
```scala
 val tweetStream = TwitterUtils.createStream(ssc, Utils.getAuth)
  .map(gson.toJson(_))

tweetStream.foreachRDD((rdd, time) => {
  val count = rdd.count()
  if (count > 0) {
    val outputRDD = rdd.repartition(partitionsEachInterval)
    outputRDD.saveAsTextFile(
      outputDirectory + "/tweets_" + time.milliseconds.toString)
    numTweetsCollected += count
    if (numTweetsCollected > numTweetsToCollect) {
      System.exit(0)
    }
  }
})
```

#### Examining the Tweets and Training a K-means Clustering Model

The example uses SparkSQL to examine the data based on the tweets. SparkSQL can load JSON files and infer the schema based on the data. The commands you pass into SparkSQL to bring back to stdout will follow pretty standard SQL syntax.

```scala
sqlContext.sql(<command>).collect().foreach(println)
```

This clustering aims to identify clusters of tweets written in the same language. We do so by vectorizing hashed bigrams of characters within each tweet to recognize common sequences of characters in languages. Here `tf` is a `HashingTF` from `mllib.feature.HashingTF`

```scala
  def featurize(s: String): Vector = {
    tf.transform(s.sliding(2).toSeq)
  }
```

And here we train a KMeans model from MLlib:
```scala
val vectors = texts.map(Utils.featurize).cache()
val model = KMeans.train(vectors, numClusters, numIterations)
sc.makeRDD(model.clusterCenters, numClusters).
    saveAsObjectFile(outputModelDir)
```

#### Realtime Classification

Finally, let's apply the model we've created in realtime! We'll:
1. load a Spark Streaming Context
1. create a Twitter DStream and grab just their `text` field
1. load the trained KMeans model
1. Choose the id of a language cluster we're interested in, and apply the model to all tweets, filtering only on that specificed cluster to see if the printed output is what we expect. 

```scala
println("Initializing Streaming Spark Context...")
val conf = new SparkConf().setAppName(this.getClass.getSimpleName)
val ssc = new StreamingContext(conf, Seconds(5))

println("Initializing Twitter stream...")
val tweets = TwitterUtils.createStream(ssc, Utils.getAuth)
val statuses = tweets.map(_.getText)

println("Initializing the the KMeans model...")
val model = new KMeansModel(ssc.sparkContext.objectFile[Vector](
    modelFile.toString).collect())

val filteredTweets = statuses
  .filter(t => model.predict(Utils.featurize(t)) == clusterNumber)
filteredTweets.print()

```

### An exercise for the reader

The first streaming model to make it into MLlib is a streaming k-means. This means that the cluster estimations can be dynamically updated as new data arrive. The algorithm uses a generalization of the mini-batch k-means update rule. For each batch of data, we assign all points to their nearest cluster, compute new cluster centers, then update each cluster. 

In this example we trained our model "in batch" on a static store of data. Try writing another class in the `streaming_twitter_cl` project that instead initializes a new `StreamingKMeans` model with random centers and the number of languages you want to classify as your number of clusters, and dynamically updates on fresh stream of Twitter ata.

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*